### 1. 安装HuggingFace 并下载模型到本地

In [ ]:
#!pip install huggingface-hub -Uqq
!pip install accelerate">=0.17.2"

In [1]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path = Path("./LLM_baichuan_model")
local_model_path.mkdir(exist_ok=True)
#model_name = "baichuan-inc/baichuan-7B"
model_name = "baichuan-inc/Baichuan-13B-Base"
#commit_hash = "ba9db8ed916eb8c4d4349d40ef7a0b6b68a0b930"

In [ ]:
#snapshot_download(repo_id=model_name, revision=commit_hash,cache_dir=local_model_path)
snapshot_download(repo_id=model_name,cache_dir=local_model_path)

### 2. 把模型拷贝到S3为后续部署做准备

In [2]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [ ]:
s3_model_prefix = "llm/models/LLM_baichuan_model_13b"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/LLM_baichuan_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

In [ ]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

In [52]:
!aws s3 ls s3://sagemaker-us-west-2-687912291502/llm/models/LLM_baichuan_model_13b/

2023-07-19 08:40:19       1519 .gitattributes
2023-07-19 08:40:19     187689 Baichuan-13B 模型社区许可协议.pdf
2023-07-19 08:40:19     200943 Community License for Baichuan-13B Model.pdf
2023-07-19 08:40:19      16415 README.md
2023-07-19 08:40:19        680 config.json
2023-07-19 08:40:19       1490 configuration_baichuan.py
2023-07-19 08:40:19        284 generation_config.json
2023-07-19 08:40:19      24547 modeling_baichuan.py
2023-07-19 08:40:19 9972281188 pytorch_model-00001-of-00003.bin
2023-07-19 08:40:19 9947421360 pytorch_model-00002-of-00003.bin
2023-07-19 08:40:19 6610200289 pytorch_model-00003-of-00003.bin
2023-07-19 08:40:19      23274 pytorch_model.bin.index.json
2023-07-19 08:40:19      21111 quantizer.py
2023-07-19 08:40:20         86 requirements.txt
2023-07-19 08:40:20        544 special_tokens_map.json
2023-07-19 08:40:20       8720 tokenization_baichuan.py
2023-07-19 08:40:20    1136765 tokenizer.model
2023-07-19 08:40:20        954 tokenizer_config.json


### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [26]:
inference_image_uri = (
    #"763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118"
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.22.1-deepspeed0.9.2-cu118"
    #f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117"
)

print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.22.1-deepspeed0.9.2-cu118


In [27]:
!mkdir -p LLM_baichuan_deploy_code

In [28]:
%%writefile LLM_baichuan_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os
from transformers import pipeline, AutoModel, AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import deepspeed


def load_model(properties):
    tensor_parallel_degree = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location,trust_remote_code=True)
    
    model = AutoModelForCausalLM.from_pretrained(  
    pretrained_model_name_or_path=model_location,  
    #device_map="auto",  
    trust_remote_code=True,
    low_cpu_mem_usage=True,    
    torch_dtype=torch.float16)
    
    model = deepspeed.init_inference(model,
          tensor_parallel={"tp_size": tensor_parallel_degree},
          dtype=model.dtype,
          replace_method='auto',
          replace_with_kernel_inject=True)
    
    local_rank = int(os.getenv("LOCAL_RANK", "0"))
    generator = pipeline(
        task="text-generation", model=model, tokenizer=tokenizer, device=local_rank, use_cache=True
    )
    ### for accelerate ####
    #generator = pipeline(
    #    task="text-generation", model=model, tokenizer=tokenizer, device_map="auto", use_cache=True
    #)
    #model.requires_grad_(False)
    #model.eval()
    return model, pipeline, tokenizer


pipeline = None
tokenizer = None
model = None


def handle(inputs: Input):
    global model,pipeline, tokenizer
    if not pipeline:
        model, pipeline, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    #input_sentences = prefix_prompt+"User: "+data["inputs"]
    input_sentences = data["inputs"]
    params = data["parameters"]
    input_ids = tokenizer.encode(input_sentences, return_tensors='pt').to('cuda')

    with torch.no_grad():  
       output_ids = model.generate(  
            input_ids,
            **params)

    response=tokenizer.decode(output_ids[0], skip_special_tokens=True)
    result = {"outputs": response}
    return Output().add_as_json(result)

Overwriting LLM_baichuan_deploy_code/model.py


#### Note: option.s3url 需要按照自己的账号进行修改

In [29]:
%%writefile LLM_baichuan_deploy_code/serving.properties
#engine=Python
engine=DeepSpeed
option.entryPoint=model.py
option.load_in_8bit=TRUE
option.tensor_parallel_degree=4
option.s3url=s3://sagemaker-us-west-2-687912291502/llm/models/LLM_baichuan_model_13b/

Overwriting LLM_baichuan_deploy_code/serving.properties


#### 注意: baichuan config 上transformers是4.29.2

In [30]:
%%writefile LLM_baichuan_deploy_code/requirements.txt
transformers==4.29.2
accelerate>=0.17.1
einops
transformers_stream_generator

Overwriting LLM_baichuan_deploy_code/requirements.txt


In [31]:
!rm model.tar.gz
!cd LLM_baichuan_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf model.tar.gz LLM_baichuan_deploy_code

LLM_baichuan_deploy_code/
LLM_baichuan_deploy_code/model.py
LLM_baichuan_deploy_code/serving.properties
LLM_baichuan_deploy_code/requirements.txt


In [32]:
s3_model_prefix = "llm/models/LLM_baichuan_model_13b" 
s3_code_prefix = "LLM-RAG/workshop/LLM_baichuan_deploy_code"
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_baichuan_deploy_code/model.tar.gz


### 4. 创建模型 & 创建endpoint

In [53]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base(f"baichuan") # Append a timestamp to the provided string
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

baichuan-2023-08-23-23-10-53-465
Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.22.1-deepspeed0.9.2-cu118
Created Model: arn:aws:sagemaker:us-west-2:687912291502:model/baichuan-2023-08-23-23-10-53-465


In [54]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"


endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.24xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 15*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:687912291502:endpoint-config/baichuan-2023-08-23-23-10-53-465-config',
 'ResponseMetadata': {'RequestId': 'fe39e0b3-a7f5-4e67-bbac-ad7caaf2a4ec',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fe39e0b3-a7f5-4e67-bbac-ad7caaf2a4ec',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '120',
   'date': 'Wed, 23 Aug 2023 23:10:54 GMT'},
  'RetryAttempts': 0}}

In [55]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-west-2:687912291502:endpoint/baichuan-2023-08-23-23-10-53-465-endpoint


#### 持续检测模型部署进度

In [ ]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Failed
Arn: arn:aws:sagemaker:us-west-2:687912291502:endpoint/baichuan-2023-08-23-23-10-53-465-endpoint
Status: Failed


### 5. 模型测试

In [ ]:
endpoint_name = "baichuan-2023-07-18-04-51-39-353-endpoint"
endpoint_name = "baichuan-2023-07-18-06-55-32-845-endpoint"
prompt = """
你是MySQL的专家。给定一个输入问题，创建一个语法正确的MySQL查询语句。
除非用户在问题中指定了要获得的特定数量的示例，否则使用LIMIT子句查询最多3个结果。您可以对结果进行排序，以返回数据库中信息量最大的数据。您必须仅查询回答问题所需的列。将每个列名用反引号（`）括起来，表示为分隔的标识符。
请注意，仅可以使用在下面这些表中看到的列名，不要查询不存在的列。此外，还要注意哪个列在哪个表中。如果问题涉及”今天”，请注意使用CURDATE()函数获取当前日期.

使用如下格式:
Question: 具体的问题
SQLQuery: 运行的sql语句
SQLResult: SQLQuery运行的结果
Answer: 最终的回答


使用如下的表:
CREATE TABLE customer (
	c_customer_sk INTEGER NOT NULL, 
	c_customer_id CHAR(16) NOT NULL, 
	c_current_cdemo_sk INTEGER, 
	c_current_hdemo_sk INTEGER, 
	c_current_addr_sk INTEGER, 
	c_first_shipto_date_sk INTEGER, 
	c_first_sales_date_sk INTEGER, 
	c_salutation CHAR(10), 
	c_first_name CHAR(20), 
	c_last_name CHAR(30), 
	c_preferred_cust_flag CHAR(1), 
	c_birth_day INTEGER, 
	c_birth_month INTEGER, 
	c_birth_year INTEGER, 
	c_birth_country VARCHAR(20), 
	c_login CHAR(13), 
	c_email_address CHAR(50), 
	c_last_review_date CHAR(10), 
	PRIMARY KEY (c_customer_sk)
)ENGINE=InnoDB DEFAULT CHARSET=utf8


CREATE TABLE web_sales (
	ws_sold_date_sk INTEGER, 
	ws_sold_time_sk INTEGER, 
	ws_ship_date_sk INTEGER, 
	ws_item_sk INTEGER NOT NULL, 
	ws_bill_customer_sk INTEGER, 
	ws_bill_cdemo_sk INTEGER, 
	ws_bill_hdemo_sk INTEGER, 
	ws_bill_addr_sk INTEGER, 
	ws_ship_customer_sk INTEGER, 
	ws_ship_cdemo_sk INTEGER, 
	ws_ship_hdemo_sk INTEGER, 
	ws_ship_addr_sk INTEGER, 
	ws_web_page_sk INTEGER, 
	ws_web_site_sk INTEGER, 
	ws_ship_mode_sk INTEGER, 
	ws_warehouse_sk INTEGER, 
	ws_promo_sk INTEGER, 
	ws_order_number INTEGER NOT NULL, 
	ws_quantity INTEGER, 
	ws_wholesale_cost DECIMAL(7, 2), 
	ws_list_price DECIMAL(7, 2), 
	ws_sales_price DECIMAL(7, 2), 
	ws_ext_discount_amt DECIMAL(7, 2), 
	ws_ext_sales_price DECIMAL(7, 2), 
	ws_ext_wholesale_cost DECIMAL(7, 2), 
	ws_ext_list_price DECIMAL(7, 2), 
	ws_ext_tax DECIMAL(7, 2), 
	ws_coupon_amt DECIMAL(7, 2), 
	ws_ext_ship_cost DECIMAL(7, 2), 
	ws_net_paid DECIMAL(7, 2), 
	ws_net_paid_inc_tax DECIMAL(7, 2), 
	ws_net_paid_inc_ship DECIMAL(7, 2), 
	ws_net_paid_inc_ship_tax DECIMAL(7, 2), 
	ws_net_profit DECIMAL(7, 2), 
	PRIMARY KEY (ws_item_sk, ws_order_number)
)ENGINE=InnoDB DEFAULT CHARSET=utf8

Question: 我需要知道销售报表中，下单金额最大的客户email地址
"""

prompts2="给我一个青海和甘肃旅游的路线，8天7晚"
prompts3="好累啊"
parameters={
    "do_sample": False,
    "top_p": 0.9,
    "temperature": 1,
    "max_new_tokens": 300,
    "repetition_penalty": 1.03
}
#response_model = smr_client.invoke_endpoint(
#            EndpointName=endpoint_name,
#            Body=json.dumps(
#            {
#                "inputs": prompt,
#                "parameters": parameters,
#            }
#            ),
#            ContentType="application/json",
#        )
#
#response_model['Body'].read().decode("utf-8")

In [ ]:
prompt="""你正在一个聊天室里和不同国家的人们聊天，你能读懂所有国家的语言，你负责通过聊天记录分析所有聊天者的性格和有效信息，具体步骤如下：
1.阅读他们的聊天记录
2.总结他们聊天里面的重要信息
3.抽象他们的人设
4.使用评分体系抽象他们之间的人际关系，然后给一个评分，范围1-10分，分越高关系越好"""

prompt=prompt+"""聊天记录如下：
                WaRGazmo : "you lucked out there buddy" 
                WarLord : "suerte? eso no existe " 
                WarLord : "soy más rápido que la luz " 
                WaRGazmo : "it exists.. or karma" 
                DirtyE1bow : "so you was a planned birth ?" 
                WaRGazmo : "thats what she said bruh" 
                WarLord : "te amo mi amor " 
                Manowarik : "Мир вам,люди добрые.." 
                kotofei : "и тебе боярин, что не подался в челядь королю)" 
                XxNORxXMithra : "God morgen folkens :) " 
                kotofei : "и прочие жители галактики " 
                XxNORxXMithra : "Ja de også forsåvidt :) " 
                Manowarik : "Котофей-это который по цепи кругом?Песни там,сказки?😆😆" 
                kotofei : "не, то дальний убогий родственник " 
                Manowarik : "Эххх..Лукоморье мимо..((" 
                kipl : "Котофей он из сказки Лиса и Котофей Иванович. " 
                kipl : "Межвидовой брак и крышевание леса" 
                kotofei : "лиса 🦊 мералиса и Котофей Иваныч " 
                leister : "😆" XxFoxyQBAxX : "po co tyle zrobiłeś?" """

response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompt,
                "parameters": parameters,
            }
            ),
            ContentType="application/json",
        )

response_model['Body'].read().decode("utf-8")


#### 清除模型Endpoint和config

In [ ]:
!aws sagemaker delete-endpoint --endpoint-name chatglm-2023-04-27-05-49-59-117-endpoint

In [ ]:
!aws sagemaker delete-endpoint-config --endpoint-config-name chatglm-2023-04-27-05-49-59-117-config